In [50]:
#!pip install bayesian-optimization  # for google collab
#!pip install git+https://github.com/slremy/netsapi --user --upgrade

In [51]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import UtilityFunction
from netsapi.challenge import *
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import mlab
from matplotlib import gridspec
%matplotlib inline
from sys import exit, exc_info, argv
from multiprocessing import Pool, current_process
import random as rand
import json
import requests
import numpy as np
import pandas as pd
import statistics
from IPython.display import clear_output
import time

from contextlib import contextmanager
import sys, os
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import math
from tqdm import tqdm

def newHoleEnv():
    #return ChallengeProveEnvironment()
    return ChallengeSeqDecEnvironment() #Initialise a New Challenge Environment to post entire policy

print("done")

done


# hyper param and init

In [52]:
env = newHoleEnv()

#hyper param
x_start = 0.0
x_end = 1.0
normalisation = 90.0 # reward normalisation

# Utility : Neural net

In [53]:

# action val in
x_start = 0.0
x_end = 1.0


In [54]:
#utility

In [55]:
def truncate_policy_to_needed_param(Allparam, needed_param):
    params = {}
    for key in needed_param:
        params[key]=Allparam[key]
    return params


def queryMyPolicy(policy):
    detailed_reward= {1:0, 2:0, 3:0, 4:0, 5:0}
    env.reset()
    for i in range (1,6):
        nextstate, reward, done, _ = env.evaluateAction([ policy[str(i)+'x'] , policy[str(i)+'y']  ] )
        detailed_reward[i] = reward/ normalisation
    
    global policies, detailed_rewards , UsedPolicy
    policies.append(policy)
    detailed_rewards.append(detailed_reward)
    UsedPolicy+=1
    return policy, detailed_reward

#  list of hm's target

# balance's target

# Train Utility

In [56]:
def fit(bo):
    bo._gp.fit(bo._space.params, bo._space.target)
    
def predict(bo,Policy):
        params =truncate_policy_to_needed_param(Policy, bo.space.keys)

        valLeftPolicy = [] 
        for val in params.values():
            valLeftPolicy.append(val)

        params = np.array( valLeftPolicy)
        params.reshape(1, -1)

        return bo._gp.predict( [params] , return_std=True)
    

## 2d_plot (balance)

## model target

# Train

# main

In [57]:
normalisation = 90.0  
takeNote = False
curPolicy = {}
curReward = 0.0

allPolicies = []


def Y1_target1(x, y):
    x = np.asscalar(x)
    y = np.asscalar(y)
    env.reset()
    action = [x , y]
    s,r,d,_ = env.evaluateAction(action)
    print(s,r,d,_, " this should be y1")
    if (takeNote):
        global curPolicy,curReward
        curPolicy = {1: [x,y]}
        curReward = r
    return r/ normalisation 

def Y1_target(x,y):
    if type(x) is np.ndarray:
        result = []
        for a,b in zip(x,y):
            reward = Y1_target1(a,b)
            result.append( reward )
        return result
    else:
        return Y1_target1(x,y)
    
    
    
    
def target1(x, y):
    x = np.asscalar(x)
    y = np.asscalar(y)
    
    action = [x , y]
    s,r,d,_ = env.evaluateAction(action)
    print(s,r,d,_)
    if (takeNote):
        global curPolicy,curReward
        curPolicy[s-1] =  [x,y]
        curReward += r
        if (d==True):
            curPolicy['Reward'] = curReward
            allPolicies.append(curPolicy)
    return r/90.0

def target(x,y):
    if type(x) is np.ndarray:
        result = []
        for a,b in zip(x,y):
            reward = target1(a,b)
            result.append( reward )
        return result
    else:
        return target1(x,y)



In [58]:
env = newHoleEnv()


bos = [] 
bos_target_out = []

#############################################TODO loop
bos.append( BayesianOptimization(f=Y1_target, pbounds={'x': (x_start, x_end), 'y': (x_start, x_end)},verbose=1))
bos_target_out.append( [1] )

bos.append( BayesianOptimization(f=target, pbounds={'x': (x_start, x_end), 'y': (x_start, x_end)} ,verbose=1))
bos_target_out.append( [2] )

bos.append( BayesianOptimization(f=target, pbounds={'x': (x_start, x_end), 'y': (x_start, x_end)} ,verbose=1))
bos_target_out.append( [3] )

bos.append( BayesianOptimization(f=target, pbounds={'x': (x_start, x_end), 'y': (x_start, x_end)} ,verbose=1))
bos_target_out.append( [4] )

bos.append( BayesianOptimization(f=target, pbounds={'x': (x_start, x_end), 'y': (x_start, x_end)} ,verbose=1))
bos_target_out.append( [5] )


# 0
bos[0].maximize(init_points=5, n_iter=0, acq='ucb', kappa=10)
bos[0].maximize(init_points=0, n_iter=8, acq='ucb', kappa=10)
bos[0].maximize(init_points=0, n_iter=2, acq='ucb', kappa=.5)
bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=.1)

# 16
bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=2.5)
bos[1].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)

# 18
bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=2.5)
bos[1].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)

# 20
for _ in range(3):
    bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=2.5)
    bos[1].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)
    bos[2].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)

# 29
for _ in range(4):
    bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=2)
    bos[1].maximize(init_points=0, n_iter=1, acq='ucb', kappa=10)
    bos[2].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)
    bos[3].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)
#45    
for _ in range(5):
    bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1.5)
    bos[1].maximize(init_points=0, n_iter=1, acq='ucb', kappa=8)
    bos[2].maximize(init_points=0, n_iter=1, acq='ucb', kappa=9)
    bos[3].maximize(init_points=0, n_iter=1, acq='ucb', kappa=10)
    
#65    
takeNote = True
for _ in range(4):
    bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
    bos[1].maximize(init_points=0, n_iter=1, acq='ucb', kappa=5)
    bos[2].maximize(init_points=0, n_iter=1, acq='ucb', kappa=6)
    bos[3].maximize(init_points=0, n_iter=1, acq='ucb', kappa=8)
    bos[4].maximize(init_points=1, n_iter=0, acq='ucb', kappa=10)
    
#85
for _ in range(2):
    bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
    bos[1].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
    bos[2].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
    bos[3].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
    bos[4].maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
#95    
bos[0].maximize(init_points=0, n_iter=1, acq='ucb', kappa=0)
bos[1].maximize(init_points=0, n_iter=1, acq='ucb', kappa=0)
bos[2].maximize(init_points=0, n_iter=1, acq='ucb', kappa=0)
bos[3].maximize(init_points=0, n_iter=1, acq='ucb', kappa=0)
bos[4].maximize(init_points=0, n_iter=1, acq='ucb', kappa=0)
#100


####################
allPolicies.sort(key=lambda x: x['Reward'], reverse=True)
        
best_policy= allPolicies[0]
del best_policy['Reward']
best_policy        
best_reward = env.evaluatePolicy(best_policy)
#print("found ", reward_AllPolicy)

print(best_policy, best_reward)

#return best_policy, best_reward


|   iter    |  target   |     x     |     y     |
-------------------------------------------------
105  Evaluations Remaining
2 47.1091440553834 False {}  this should be y1
104  Evaluations Remaining
2 22.14511125608262 False {}  this should be y1
103  Evaluations Remaining
2 -36.065438036316515 False {}  this should be y1
102  Evaluations Remaining
2 30.73864578473492 False {}  this should be y1
101  Evaluations Remaining
2 7.9095933327375825 False {}  this should be y1
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
100  Evaluations Remaining
2 60.90957750458907 False {}  this should be y1
|  6        |  0.6768   |  0.6969   |  0.184    |
99  Evaluations Remaining
2 22.97014034523438 False {}  this should be y1
98  Evaluations Remaining
2 15.106956390123337 False {}  this should be y1
97  Evaluations Remaining
2 92.08013377763152 False {}  this should be y1
|  9        |  1.023    |  0.0      |  1.0      |
96  Evaluations Remaining

61  Evaluations Remaining
5 -31.263634731509224 False {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
60  Evaluations Remaining
2 91.08181964761937 False {}  this should be y1
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
59  Evaluations Remaining
3 28.622251412175693 False {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
58  Evaluations Remaining
4 106.71266491303228 False {}
|  8        |  1.186    |  0.0      |  1.0      |
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
57  Evaluations Remaining
5 99.78591269817188 False {}
|  5        |  1.109    |  1.0      |  0.0      |
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
56  Evaluations Remaining
2 11.865328500468362 False {}  this should be y1
|   iter    |  target   |  

23  Evaluations Remaining
4 57.671099380266945 False {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
22  Evaluations Remaining
5 46.38366212455419 False {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
21  Evaluations Remaining
6 14.277068692378972 True {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
20  Evaluations Remaining
2 40.53748753688841 False {}  this should be y1
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
19  Evaluations Remaining
3 37.87333055655532 False {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
18  Evaluations Remaining
4 49.69620439567016 False {}
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
17  Evaluations Remaining
5 2.85329339836994 False